## Evaluate particulate Mn model concentrations with observations 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import netCDF4 as nc
import pandas as pd
import modules.mapping_functions as mf
import glob
from datetime import datetime
import xarray as xr

%matplotlib inline

#### Parameters:

In [ ]:
obs_color   = '#e54141'
model_color = '#33c7ff'

#### Load files

In [ ]:
# GEOTRACES station information based on Colombo et al., 2020:
GEOTRACES = pd.read_csv('/ocean/brogalla/GEOTRACES/data/Pb-paper-data.csv')
stn_names = GEOTRACES['Station names'].dropna().astype('str')
Manuel_lons = GEOTRACES['station lon'].astype('float').dropna().values
Manuel_lats = GEOTRACES['Station lat'].astype('float').dropna().values

In [ ]:
# Template file structure:
r = nc.Dataset('/data/brogalla/run_storage/Mn-reference-202204/ANHA12_ref-2015_20220422/ANHA12_EXH006_2015_monthly.nc')
lon_model = np.array(r.variables['nav_lon'])[:,:]
lat_model = np.array(r.variables['nav_lat'])[:,:]
deptht    = np.array(r.variables['deptht'])

#### Functions:

In [ ]:
def calculate_average(folder_2015):
    # Calculate average over data files during cruise period:
    # 2015 GEOTRACES: 9 August - 15 September
    files_2015      = np.sort(glob.glob(f'{folder_2015}ANHA12_EXH006_5d_20150101_20151231*'))
    start_date_2015 = datetime.strptime(f'2015-08-09', '%Y-%m-%d')
    end_date_2015   = datetime.strptime(f'2015-09-15', '%Y-%m-%d')

    file_list_2015 = []
    for file in files_2015:
        if (datetime.strptime(file.split('/')[-1][42:50],'%Y%m%d') >= start_date_2015) & \
                    (datetime.strptime(file.split('/')[-1][51:59],'%Y%m%d') < end_date_2015):
            file_list_2015.append(file)

    dmn_2015 = np.zeros((50,700,640))
    days = 0
    for file in file_list_2015:
        ds = xr.open_dataset(f"{file}")
        dmn_2015 = dmn_2015 + ds['dissolmn'].values[0,:,:,:]

    mn_dis_2015 = dmn_2015 / len(file_list_2015)
    
    return mn_dis_2015

In [ ]:
def plot_profiles(name, ax, model_depth, model_dmn_ref, model_dmn_bio, ylim=(100,0)): 
    ax.invert_yaxis()
    ax.xaxis.tick_top()
    ax.set_xlim(-0.31, 0.3)
    ax.plot([0,0],[ylim[0],ylim[1]],'-k')
    ax.set_xticks([-0.3,-0.2,-0.1,0,0.1,0.2,0.3])
    ax.set_xticklabels([-0.3,'', '', 0,'','',0.3])
    ax.plot((model_dmn_bio-model_dmn_ref)*1e9, model_depth, c=model_color)
    
    try:
        ax.set_ylim(ylim[0],ylim[1])
        ax.text(0.05, ylim[0]*0.9, name, fontsize=5.5)
    except:
        print('empty obs depth array')
        
    ax.tick_params(axis='both', labelsize=6)
        
    return

#### Figure

In [ ]:
dmn_ref_2015 = calculate_average('/data/brogalla/run_storage/Mn-reference-202204/ANHA12_ref-2015_20220422/')
dmn_bio_2015 = calculate_average('/data/brogalla/run_storage/Mn-bio-202204/ANHA12_bio-2015_20220422/')

In [ ]:
stn_list = ['CAA1','CAA2','CAA3','CAA4','CAA5','CAA6','CAA7','CAA8','CAA9']

fig, ax = plt.subplots(2,5,figsize=(4.5, 3.7*0.8), dpi=300)
for n, stn in enumerate(stn_list):
    i, j = mf.find_indeces_vector(Manuel_lons[stn_names==stn], Manuel_lats[stn_names==stn], lon_model, lat_model)
    
    if n < 4:     
        plot_profiles(stn, ax[0,n], deptht, \
                      dmn_ref_2015[:,int(i),int(j)], dmn_bio_2015[:,int(i),int(j)], \
                      ylim=(100,0))
    else:     
        plot_profiles(stn, ax[1,n-4], deptht, \
                      dmn_ref_2015[:,int(i),int(j)], dmn_bio_2015[:,int(i),int(j)], \
                      ylim=(100,0))
        
ax[0,4].axis("off")
fig.text(0.25,1.02,'dMn change (bio minus reference experiment) [nM]', fontsize=6)
    
ax[0,0].set_ylabel('Depth [m]', fontsize=6)
ax[1,0].set_ylabel('Depth [m]', fontsize=6)

plt.tight_layout(pad=0.2, w_pad=-0.3, h_pad=0.2)

fig.savefig('/ocean/brogalla/GEOTRACES/figures/paper1-202204/S19-bio-exp.jpg', \
               bbox_inches='tight', dpi=300)
fig.savefig('/ocean/brogalla/GEOTRACES/figures/paper1-202204/S19-bio-exp.svg', \
               format='svg', bbox_inches='tight', dpi=300)